<a href="https://colab.research.google.com/github/clashleyuncc/Intro-To-ML/blob/main/HW7P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# P1a
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
import datetime
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

#use relu
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    #self.act1 = nn.Tanh()
    #self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    #self.act2 = nn.Tanh()
    #self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8 * 8 * 8, 32)
    #self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
    out = out.view(-1, 8 * 8 * 8)
    #out = self.act3(self.fcl(out))
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      #batch_size = imgs.shape[0]
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1 or epoch % 10 == 0:
      print('{} Seconds, Epoch {}, Training Loss {}'.format(
          time.time()-start_time, epoch,
          loss_train / len(train_loader)))

from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                  transform=transforms.ToTensor())

#transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

#model.to(device)
model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

  with torch.no_grad():
    for imgs, labels in val_loader:
      imgs, labels = imgs.to(device), labels.to(device)
      #batch_size = imgs.shape[0]
      outputs = model(imgs)
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correct += int((predicted == labels).sum())
  print("Accuracy (): (:.2f)".format(name, correct/total))

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

validate(model, train_loader, val_loader)

print(f"Training Time: {time.time()-start_time} seconds")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
13.024902820587158 Seconds, Epoch 1, Training Loss 2.24947484771309
94.4819986820221 Seconds, Epoch 10, Training Loss 1.4023154464829
184.72989320755005 Seconds, Epoch 20, Training Loss 1.215971731392624
275.0122880935669 Seconds, Epoch 30, Training Loss 1.0884805005377212
364.2808926105499 Seconds, Epoch 40, Training Loss 1.002556138331323
454.0298388004303 Seconds, Epoch 50, Training Loss 0.9346117565546499
543.470997095108 Seconds, Epoch 60, Training Loss 0.885259765836284
632.9128577709198 Seconds, Epoch 70, Training Loss 0.8477149313825476
722.5146992206573 Seconds, Epoch 80, Training Loss 0.8187015367209759
811.8959851264954 Seconds, Epoch 90, Training Loss 0.7928898269334412
901.010904788971 Seconds, Epoch 100, Training Loss 0.7707575494066223


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
# P1b
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
import datetime
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

#use relu
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8 * 8 * 8, 32)
    self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
    out = out.view(-1, 8 * 8 * 8)
    out = self.act3(self.fcl(out))
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out


def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      #batch_size = imgs.shape[0]
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1 or epoch % 10 == 0:
      print('{} Seconds, Epoch {}, Training Loss {}'.format(
          time.time()-start_time, epoch,
          loss_train / len(train_loader)))

#from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())

#transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

#model.to(device)
model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

  with torch.no_grad():
    for imgs, labels in val_loader:
      #imgs, labels = imgs.to(device), labels.to(device)
      #batch_size = imgs.shape[0]
      outputs = model(imgs)
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correct += int((predicted == labels).sum())
  print("Accuracy (): (:.2f)".format(name, correct/total))

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

validate(model, train_loader, val_loader)

print(f"Training Time: {time.time()-start_time} seconds")